<div style="display: flex; justify-content: space-between; align-items: center; padding: 8px 16px; background: #F8F9FA; border-bottom: 2px solid #E0E0E0; margin: 0; line-height: 1;">
    <div style="font-size: 14px; color: #666;">
        <span style="font-weight: bold; color: #333;">{SOURCE_PLATFORM} → Databricks Migration</span>
        <span style="margin-left: 8px; color: #999;">|</span>
        <span style="margin-left: 8px;">03 - Execute</span>
    </div>
    <div style="display: flex; align-items: center; gap: 8px;">
        <img src="https://cdn.simpleicons.org/snowflake/29B5E8" width="24" height="24"/>
        <span style="color: #999; font-size: 16px;">→</span>
        <img src="https://cdn.simpleicons.org/databricks/FF3621" width="24" height="24"/>
    </div>
</div>


<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>

# SQL and Code Conversion

## Overview

After migrating schemas and data, you must convert your SQL queries, stored procedures, and application code to work with Databricks. This lesson covers SQL dialect differences, conversion patterns, and best practices for porting business logic to Databricks SQL and Spark.

## Learning Objectives

By the end of this lesson, you will be able to:
- Identify SQL syntax differences between {SOURCE_PLATFORM} and Databricks SQL
- Convert common SQL patterns (date functions, string functions, CTEs)
- Port stored procedures and UDFs to Databricks
- Use AI assistants for automated code conversion
- Validate converted queries for correctness and performance

## SQL Conversion Process

SQL conversion follows a structured approach from inventory through validation.

<br />
<div class="mermaid">
flowchart LR
    INV["<b>Inventory</b><br/><i>Catalog all SQL assets</i>"]
    ASSESS["<b>Assess</b><br/><i>Identify conversion patterns</i>"]
    CONVERT["<b>Convert</b><br/><i>Translate syntax</i>"]
    VALIDATE["<b>Validate</b><br/><i>Test correctness</i>"]
    OPTIMIZE["<b>Optimize</b><br/><i>Tune performance</i>"]
    INV --> ASSESS --> CONVERT --> VALIDATE --> OPTIMIZE
    style INV fill:#e3f2fd,stroke:#1976d2
    style ASSESS fill:#e3f2fd,stroke:#1976d2
    style CONVERT fill:#fff3e0,stroke:#ff9800
    style VALIDATE fill:#e8f5e9,stroke:#4caf50
    style OPTIMIZE fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Asset Types to Convert

| Asset Type | Conversion Target | Complexity |
|------------|-------------------|------------|
| **SELECT queries** | Databricks SQL | Low-Medium |
| **Views** | Databricks Views | Low |
| **Stored procedures** | Python UDFs, Notebooks, or SQL Procedures | Medium-High |
| **User-defined functions** | Databricks UDFs (SQL or Python) | Medium |
| **Scheduled queries** | Lakeflow Jobs | Medium |
| **BI tool queries** | Databricks SQL Warehouse | Low |

## Common SQL Syntax Differences

While Databricks SQL is ANSI-compliant, there are syntax differences from {SOURCE_PLATFORM} that require conversion.

### Date and Time Functions

| {SOURCE_PLATFORM} | Databricks SQL | Notes |
|-------------------|----------------|-------|
| `DATEADD(day, 7, date_col)` | `date_add(date_col, 7)` | Different argument order |
| `DATEDIFF(day, d1, d2)` | `datediff(d2, d1)` | Arguments reversed |
| `GETDATE()` | `current_timestamp()` | Standard function |
| `TO_DATE(str, 'YYYY-MM-DD')` | `to_date(str, 'yyyy-MM-dd')` | Java date format patterns |
| `DATEPART(month, date_col)` | `month(date_col)` | Direct function |
| `TRUNC(date_col, 'MM')` | `date_trunc('month', date_col)` | Different syntax |

### String Functions

| {SOURCE_PLATFORM} | Databricks SQL | Notes |
|-------------------|----------------|-------|
| `LEN(str)` | `length(str)` | Function name |
| `CHARINDEX(sub, str)` | `locate(sub, str)` | Function name |
| `SUBSTRING(str, 1, 5)` | `substring(str, 1, 5)` | Same syntax |
| `ISNULL(col, default)` | `coalesce(col, default)` | Use standard COALESCE |
| `NVL(col, default)` | `coalesce(col, default)` | Use standard COALESCE |
| `CONCAT(a, b, c)` | `concat(a, b, c)` | Same syntax |
| `a \|\| b` | `concat(a, b)` | Concat operator varies |

### Numeric and Type Functions

| {SOURCE_PLATFORM} | Databricks SQL | Notes |
|-------------------|----------------|-------|
| `CONVERT(INT, col)` | `cast(col AS INT)` | Use CAST |
| `TRY_CAST(col AS INT)` | `try_cast(col AS INT)` | Same syntax |
| `ROUND(col, 2)` | `round(col, 2)` | Same syntax |
| `MOD(a, b)` | `a % b` or `mod(a, b)` | Either works |
| `RANDOM()` | `rand()` | Function name |

### Conditional and NULL Handling

<div class="code-block" data-language="sql">
-- {SOURCE_PLATFORM}: DECODE
-- SELECT DECODE(status, 'A', 'Active', 'I', 'Inactive', 'Unknown')

-- Databricks SQL: Use CASE WHEN
SELECT CASE status
    WHEN 'A' THEN 'Active'
    WHEN 'I' THEN 'Inactive'
    ELSE 'Unknown'
END AS status_name
FROM table;

-- {SOURCE_PLATFORM}: IIF
-- SELECT IIF(amount > 0, 'Credit', 'Debit')

-- Databricks SQL: Use IF or CASE
SELECT IF(amount > 0, 'Credit', 'Debit') AS transaction_type
FROM table;

-- NULL handling
SELECT 
    coalesce(col1, col2, 'default') AS result,  -- First non-null
    nullif(col1, col2) AS null_if_equal,         -- NULL if equal
    nvl2(col1, 'has value', 'is null') AS desc   -- nvl2 supported
FROM table;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Window Functions

Window functions are largely compatible, but some syntax differences exist.

<div class="code-block" data-language="sql">
-- ROW_NUMBER, RANK, DENSE_RANK - same syntax
SELECT 
    customer_id,
    order_date,
    amount,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date DESC) as rn,
    SUM(amount) OVER (PARTITION BY customer_id ORDER BY order_date 
                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_total
FROM orders;

-- LAG and LEAD
SELECT 
    order_date,
    amount,
    LAG(amount, 1, 0) OVER (ORDER BY order_date) as prev_amount,
    LEAD(amount, 1, 0) OVER (ORDER BY order_date) as next_amount
FROM orders;

-- FIRST_VALUE, LAST_VALUE
SELECT 
    customer_id,
    FIRST_VALUE(order_date) OVER (PARTITION BY customer_id ORDER BY order_date) as first_order,
    LAST_VALUE(order_date) OVER (
        PARTITION BY customer_id ORDER BY order_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) as last_order
FROM orders;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## Stored Procedure Conversion

Stored procedures from {SOURCE_PLATFORM} can be converted to multiple Databricks constructs depending on complexity.

<br />
<div class="mermaid">
flowchart TD
    SP["<b>Stored Procedure</b><br/><i>From {SOURCE_PLATFORM}</i>"]
    Q1["<b>Complex logic?</b>"]
    SP --> Q1
    Q1 -->|"Simple"| VIEW["<b>View or SQL Statement</b><br/><i>Best for simple transformations</i>"]
    Q1 -->|"Moderate"| SQLPROC["<b>Databricks SQL Procedure</b><br/><i>Procedural SQL</i>"]
    Q1 -->|"Complex"| PYTHON["<b>Python UDF or Notebook</b><br/><i>Full programming capability</i>"]
    style SP fill:#e3f2fd,stroke:#1976d2
    style Q1 fill:#f5f5f5,stroke:#333
    style VIEW fill:#e8f5e9,stroke:#4caf50
    style SQLPROC fill:#e8f5e9,stroke:#4caf50
    style PYTHON fill:#fff3e0,stroke:#ff9800
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### Conversion Options

| Source Pattern | Target Option | When to Use |
|----------------|---------------|-------------|
| Simple query wrapper | View | Read-only, no parameters |
| Parameterized query | SQL Procedure | Simple control flow |
| Complex ETL logic | Python Notebook | Loops, conditionals, external calls |
| Reusable function | SQL or Python UDF | Called from queries |

### SQL Procedure Example

<div class="code-block" data-language="sql">
-- Databricks SQL Procedure
CREATE OR REPLACE PROCEDURE catalog.schema.refresh_summary(
    IN start_date DATE,
    IN end_date DATE
)
LANGUAGE SQL
AS $$
BEGIN
    -- Clear existing data for date range
    DELETE FROM catalog.schema.daily_summary
    WHERE summary_date BETWEEN start_date AND end_date;
    
    -- Insert fresh aggregations
    INSERT INTO catalog.schema.daily_summary
    SELECT 
        order_date as summary_date,
        COUNT(*) as order_count,
        SUM(amount) as total_amount
    FROM catalog.schema.orders
    WHERE order_date BETWEEN start_date AND end_date
    GROUP BY order_date;
END
$$;

-- Call the procedure
CALL catalog.schema.refresh_summary('2024-01-01', '2024-01-31');
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## User-Defined Functions (UDFs)

UDFs can be created in SQL or Python, depending on the complexity of the logic.

<div class="code-block" data-language="sql">
-- SQL UDF: Simple scalar function
CREATE OR REPLACE FUNCTION catalog.schema.calculate_tax(amount DOUBLE, rate DOUBLE)
RETURNS DOUBLE
RETURN amount * rate;

-- Usage
SELECT order_id, amount, calculate_tax(amount, 0.08) as tax
FROM orders;
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-sql.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'sql';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

### Python UDF Example

<div class="code-block" data-language="python">
# Python UDF for complex logic
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

# Define the function
def clean_phone_number(phone):
    if phone is None:
        return None
    # Remove all non-numeric characters
    digits = re.sub(r'\D', '', phone)
    # Format as (XXX) XXX-XXXX
    if len(digits) == 10:
        return f"({digits[:3]}) {digits[3:6]}-{digits[6:]}"
    return digits

# Register as UDF
clean_phone_udf = udf(clean_phone_number, StringType())

# Apply to DataFrame
df_cleaned = df.withColumn("phone_formatted", clean_phone_udf("phone_raw"))
</div>

<link href="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/themes/prism.min.css" rel="stylesheet" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/prism.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/prism/1.29.0/components/prism-python.min.js"></script>

<script>
(function() {
    document.querySelectorAll('.code-block').forEach(function(block) {
        var lang = block.getAttribute('data-language') || 'python';
        var code = block.textContent.trim();
        var id = 'code-' + Math.random().toString(36).substr(2, 9);
        
        block.innerHTML = 
            '<div style="position:relative;margin:16px 0;">' +
                '<button class="copy-btn" style="position:absolute;top:8px;right:8px;padding:4px 12px;font-size:12px;background:#ddd;color:#333;border:1px solid #ccc;border-radius:4px;cursor:pointer;z-index:10;">Copy</button>' +
                '<pre style="background:#f8f8f8;border-radius:8px;padding:16px;padding-top:40px;overflow-x:auto;margin:0;border:1px solid #e0e0e0;"><code id="' + id + '" class="language-' + lang + '" style="font-family:Consolas,Monaco,monospace;font-size:14px;"></code></pre>' +
            '</div>';
        
        var codeEl = document.getElementById(id);
        codeEl.textContent = code;
        Prism.highlightElement(codeEl);
        
        block.querySelector('.copy-btn').onclick = function() {
            var t = document.createElement('textarea');
            t.value = code;
            document.body.appendChild(t);
            t.select();
            document.execCommand('copy');
            document.body.removeChild(t);
            this.textContent = '✓ Copied!';
            setTimeout(() => this.textContent = 'Copy', 2000);
        };
    });
})();
</script>

## AI-Assisted Conversion

Leverage AI assistants to accelerate SQL conversion. Databricks Assistant and Lakebridge provide automated conversion capabilities.

<br />
<div class="mermaid">
flowchart LR
    subgraph INPUT["Input"]
        SOURCE["<b>{SOURCE_PLATFORM} SQL</b><br/><i>Original query</i>"]
    end
    subgraph AI["AI Conversion"]
        ASSIST["<b>Databricks Assistant</b><br/><i>or Lakebridge</i>"]
    end
    subgraph OUTPUT["Output"]
        TARGET["<b>Databricks SQL</b><br/><i>Converted query</i>"]
        REVIEW["<b>Human Review</b><br/><i>Validate & optimize</i>"]
    end
    SOURCE --> ASSIST --> TARGET --> REVIEW
    style INPUT fill:#e3f2fd,stroke:#1976d2
    style AI fill:#fff3e0,stroke:#ff9800
    style OUTPUT fill:#e8f5e9,stroke:#4caf50
</div>
<script type="module"> import mermaid from "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs"; mermaid.initialize({ startOnLoad: true, theme: "default" }); </script>

### AI Conversion Tools

| Tool | Capability | Best For |
|------|------------|----------|
| **Databricks Assistant** | Interactive SQL conversion | Ad-hoc queries, learning |
| **Lakebridge** | Batch SQL conversion | Large-scale migrations |
| **GitHub Copilot** | Code completion | Python/Spark code |

<div style="border-left: 4px solid #ff9800; background: #fff3e0; padding: 16px 20px; border-radius: 4px; margin: 16px 0;">
    <div style="display: flex; align-items: flex-start; gap: 12px;">
        <span style="font-size: 24px;">⚠️</span>
        <div>
            <strong style="color: #e65100; font-size: 1.1em;">Always Validate AI-Generated Code</strong>
            <p style="margin: 8px 0 0 0; color: #333;">
                AI assistants accelerate conversion but may introduce subtle errors. Always test converted queries against known results before deploying to production.
            </p>
        </div>
    </div>
</div>

## Validation Strategy

Validate converted queries by comparing results against the original source system.

### Validation Approach

| Check | Method | Expected Outcome |
|-------|--------|------------------|
| **Row count** | `SELECT COUNT(*)` | Exact match |
| **Column values** | Sample comparison | Match on test data |
| **Aggregates** | `SUM`, `AVG`, `MIN`, `MAX` | Match (within tolerance) |
| **Edge cases** | NULLs, empty strings, boundaries | Same behavior |
| **Performance** | Query explain plan | Acceptable execution time |

### Regression Testing

1. **Create test datasets** - Representative samples with known edge cases
2. **Run both versions** - Execute source and target queries
3. **Compare results** - Diff row counts, checksums, sample rows
4. **Document differences** - Expected changes vs. bugs
5. **Iterate** - Fix issues and re-test

## Summary

### Conversion Checklist

- [ ] Inventory all SQL assets (queries, views, procedures, UDFs)
- [ ] Map function/syntax differences to Databricks equivalents
- [ ] Convert simple queries first, validate
- [ ] Port stored procedures to appropriate targets
- [ ] Create UDFs for reusable business logic
- [ ] Validate all converted code against test data
- [ ] Optimize for Databricks (use Delta features, proper data types)

### Key Principles

| Principle | Why It Matters |
|-----------|----------------|
| **Use standard SQL** | ANSI SQL works across platforms; avoid proprietary syntax |
| **Leverage AI assistance** | Accelerate conversion, but always validate |
| **Test extensively** | Edge cases often reveal subtle conversion issues |
| **Optimize for Delta** | Use `MERGE`, partitioning, and clustering effectively |

### Next Steps

With SQL converted, proceed to orchestration:

- [**3.5 - Pipeline and Orchestration**]($./3.5 - Pipeline and Orchestration) - Rebuild workflows and scheduling

<div style="color: #FF3621; font-weight: bold; font-size: 2em; margin-bottom: 12px;">COURSE DEVELOPER (remove before publishing)</div>

### Template Customization

**Placeholders to replace:**
- `{SOURCE_PLATFORM}` - Source platform name (Snowflake, BigQuery, Redshift, Teradata)

**Platform-specific additions required:**
- Add platform-specific function mapping tables (each platform has unique functions)
- Include platform-specific stored procedure conversion examples
- Add platform-specific date/time format patterns
- Document platform-specific SQL dialect quirks

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>
